In [1]:
import pandas as pd

In [27]:
casos = pd.DataFrame({'fecha': ['2017-12-14', '2016-03-12', '2017-11-12', '2016-04-02', '2018-10-03', '2017-10-02', '2018-01-02', '2018-03-02', '2018-02-02', '2018-12-02', '2017-04-04', '2018-01-01', '2017-11-02'],
                        'id_caso': [0,1,2,3,4,5,6,7,8,9,10,11,12],
                        'descripcion': ['C0', 'C1', 'C2', 'C3','c4','5','6','7','8','9','10','11','12'],
                        'estado_caso': [1, 2, 1,2,1,1,1,2,1,2,3,1,1],
                        'categoria': ['A', 'D', 'C', 'D','A','C','A','D','D','A','C','A','C'],
                        'latitud': ['D0', 'D1', 'D2', 'D3','D0', 'D1', 'D2', 'D3','D0', 'D1', 'D2', 'D3','a'],
                        'longitud': ['D0', 'D1', 'D2', 'D3','D0', 'D1', 'D2', 'D3','D0', 'D1', 'D2', 'D3','s']},
                        index=range(0,13))

In [28]:
batman= pd.DataFrame({'id_caso': [0,1,2,3,4,5,6,7,8,9,10,11,12],
                     'respuesta':[1,0,1,1,0,1,0,0,0,1,1,0,1]})

In [29]:
casos.head(3)

,categoria,descripcion,estado_caso,fecha,id_caso,latitud,longitud
0,A,C0,1,2017-12-14,0,D0,D0
1,D,C1,2,2016-03-12,1,D1,D1
2,C,C2,1,2017-11-12,2,D2,D2


In [30]:
batman.head(3)

,id_caso,respuesta
0,0,1
1,1,0
2,2,1


In [31]:
casos = casos[['fecha','id_caso','categoria','estado_caso']]

In [32]:
casos = casos[(casos['fecha']>'2017-08-00')&(casos['fecha']<'2018-04-00')]

In [33]:
casos['fecha'] = casos.apply(lambda x: x['fecha'].split('-')[0],axis=1)

In [34]:
casos

,fecha,id_caso,categoria,estado_caso
0,2017,0,A,1
2,2017,2,C,1
5,2017,5,C,1
6,2018,6,A,1
7,2018,7,D,2
8,2018,8,D,1
11,2018,11,A,1
12,2017,12,C,1


In [35]:
#casos['trimestre']= ['actual' for x in casos['fecha']]

In [36]:
casos

,fecha,id_caso,categoria,estado_caso
0,2017,0,A,1
2,2017,2,C,1
5,2017,5,C,1
6,2018,6,A,1
7,2018,7,D,2
8,2018,8,D,1
11,2018,11,A,1
12,2017,12,C,1


In [37]:
casos['trimestre']= casos.apply(lambda x: 'actual' if x['fecha']=='2018' else 'anterior', axis=1)

In [38]:
casos


,fecha,id_caso,categoria,estado_caso,trimestre
0,2017,0,A,1,anterior
2,2017,2,C,1,anterior
5,2017,5,C,1,anterior
6,2018,6,A,1,actual
7,2018,7,D,2,actual
8,2018,8,D,1,actual
11,2018,11,A,1,actual
12,2017,12,C,1,anterior


In [39]:
casosAgrupados = casos.groupby(by=['categoria', 'trimestre']).apply(lambda x: (x[x['estado_caso']==1].count()/x['estado_caso'].count()))

In [40]:
casosAgrupados

fecha  id_caso  categoria  estado_caso  trimestre
categoria trimestre                                                   
A         actual       1.0      1.0        1.0          1.0        1.0
          anterior     1.0      1.0        1.0          1.0        1.0
C         anterior     1.0      1.0        1.0          1.0        1.0
D         actual       0.5      0.5        0.5          0.5        0.5

In [41]:
casosAgrupados=casosAgrupados.unstack()

In [42]:
casosAgrupados

fecha          id_caso          categoria          estado_caso  \
trimestre actual anterior  actual anterior    actual anterior      actual   
categoria                                                                   
A            1.0      1.0     1.0      1.0       1.0      1.0         1.0   
C            NaN      1.0     NaN      1.0       NaN      1.0         NaN   
D            0.5      NaN     0.5      NaN       0.5      NaN         0.5   

                   trimestre           
trimestre anterior    actual anterior  
categoria                              
A              1.0       1.0      1.0  
C              1.0       NaN      1.0  
D              NaN       0.5      NaN

In [45]:
tasas =casosAgrupados.rename(columns={'trimestre':'tasa'})['tasa']

In [46]:
tasas

trimestre,actual,anterior
categoria,,
A,1.0,1.0
C,NaN,1.0
D,0.5,NaN


In [47]:
tasas = tasas.fillna(0)

In [48]:
cate = tasas[tasas['actual']>(tasas['anterior']+tasas['anterior']*0.1)]

In [49]:
cate

trimestre,actual,anterior
categoria,,
D,0.5,0.0


In [50]:
casos = casos[(casos['categoria']=='A')|(casos['categoria']=='D')]

In [51]:
casos

,fecha,id_caso,categoria,estado_caso,trimestre
0,2017,0,A,1,anterior
6,2018,6,A,1,actual
7,2018,7,D,2,actual
8,2018,8,D,1,actual
11,2018,11,A,1,actual


In [52]:
casos=casos[['categoria','id_caso']]

In [53]:
casos

,categoria,id_caso
0,A,0
6,A,6
7,D,7
8,D,8
11,A,11


In [54]:
casosbatman= pd.merge(casos,batman,on='id_caso',how='left')

In [55]:
casosbatman

,categoria,id_caso,respuesta
0,A,0,1
1,A,6,0
2,D,7,0
3,D,8,0
4,A,11,0


In [56]:
cate = casosbatman.groupby('categoria').apply(lambda x: x[x['respuesta']==1].count()/x.count())

In [58]:
cate = cate[['respuesta']].rename(columns={'respuesta':'tasa'})

In [59]:
cate

,tasa
categoria,
A,0.333333
D,0.000000
